######  Note: This script can use directly for Predication just replace test_data_file as your test file

In [3]:
# Import all used packages
import pandas as pd

import datetime
import time
import numpy as np
import matplotlib
matplotlib.use('nbagg') # for more intractive visualization like zoom out
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
import xgboost as xgb
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

In [4]:
test_data_file = "../data-set/yellow_tripdata_2015-01.csv"
kmean_model_file = "../model/kmean_cluster_model.sav"
lgbm_model_file = "../model/lgbm_model.sav"

In [5]:
test_data = pd.read_csv(test_data_file)

In [6]:
#jan_data = all_data.loc[all_data.tpep_pickup_datetime.str.startswith("2019-1")]
print("test data row and columns informantion")
print(test_data.shape)
print(test_data.columns)

test data row and columns informantion
(12748986, 19)
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')


In [7]:
#Conver python date formate type into unix time format to deal better way as unix's time is presents in Seconds
test_data["pickup_time_unix"] = test_data.tpep_pickup_datetime.apply(lambda x : time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))
test_data["dropoff_time_unix"] = test_data.tpep_dropoff_datetime.apply(lambda x : time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple()))

#Make New Trip Duration feature using pickup and dropoff time
test_data["trip_duration"] = (test_data['dropoff_time_unix'].astype(int) - test_data['pickup_time_unix'].astype(int))/float(60)

#Make New Trip Speed feature for analysis
test_data["speed"] = 60*(test_data["trip_distance"]/test_data["trip_duration"])

In [9]:
# if ""../model/kmean_cluster_model.sav" not present in model folder you can re run data_cleaning.ipynb script and generate it
with open(kmean_model_file,'rb') as file_obj:
    kmeans = pickle.load(file_obj)
    test_data['pickup_cluster'] = kmeans.predict(test_data[['pickup_latitude', 'pickup_longitude']])

In [10]:
#Refer:https://www.unixtimestamp.com/
start_unix_time = 1420070400
#1575158400 : 2019-01-12 00:00:00

test_data["10_min_bin"] = test_data["pickup_time_unix"].apply(lambda x : int(abs((x-start_unix_time)/(60*10))))
test_groupby_10m = test_data[["pickup_cluster","10_min_bin","trip_distance"]].groupby(["pickup_cluster","10_min_bin"]).count()


In [11]:
test_groupby_10m.reset_index(inplace=True)
test_groupby_10m.rename(columns = {"trip_distance":"trip_count"},inplace=True)
test_groupby_10m["trip_count"]= test_groupby_10m.trip_count.astype(float)
test_groupby_10m.head()

,pickup_cluster,10_min_bin,trip_count
0,0,0,72.0
1,0,1,70.0
2,0,2,66.0
3,0,3,72.0
4,0,4,90.0


In [14]:
print("Befor Zero filling test set size : "+str(len(test_groupby_10m)))
test_groupby_10m
new_data_dict =[]
data_index = list(test_groupby_10m.groupby(["pickup_cluster","10_min_bin"]).count().index.values)
for clust in range(30):
    for slot in range(2735):
        if not (clust,slot) in data_index:
            new_data_dict.append([clust,slot,0])

new_data = pd.DataFrame(new_data_dict, columns= test_groupby_10m.columns)
test_groupby_10m = test_groupby_10m.append(new_data,ignore_index=True)
print("After Zero filling test set size : "+str(len(test_groupby_10m)))

Befor Zero filling test set size : 131349
After Zero filling test set size : 131640


In [15]:
def get_features(clusters):
    val_feature = []
    val_label = []
    for grp, val_df in clusters:
        val_df.sort_values("10_min_bin",inplace = True)
        val_feature.extend(np.array([ list(val_df.trip_count[i:i+5]) for i in range(len(val_df)-6)]))
        val_label.extend( [val_df.trip_count.iloc[i+6] for i in range(len(val_df)-6)])
    return val_feature, val_label
    

In [17]:
x_test, y_test = get_features(test_groupby_10m.groupby("pickup_cluster"))

In [18]:
#mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = np.mean(y_true)
    return np.mean(np.abs((y_true - y_pred) /y_true))*100

In [19]:
with open(lgbm_model_file,'rb') as file_obj:
    lgbm = pickle.load(file_obj)

In [21]:
y_pred = lgbm.predict(x_test)
lgbm_test_predictions = [round(value) for value in y_pred]

lgbm_mape_test = mean_absolute_percentage_error(y_test,lgbm_test_predictions)

print("Light GBM Regressor --> Test MAPE : "+str(lgbm_mape_test))

Light GBM Regressor --> Test MARE : 72.12973204005037


###### predicted values are in "lgbm_test_predictions" variable can see